### ETH Whale Activity ML Pipeline

- Setup & Configuration

In [1]:
import os
import time
import pickle
import requests
import warnings
import json
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix, make_scorer
)

try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    HAS_IMBLEARN = True
except ImportError:
    HAS_IMBLEARN = False
    print("⚠️ imbalanced-learn not installed")

try:
    from xgboost import XGBClassifier
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False
    print("⚠️ XGBoost not installed")

from dotenv import load_dotenv

warnings.filterwarnings('ignore')

⚠️ imbalanced-learn not installed


- Loading and Configuring Environmental Varriables

In [2]:
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_WHALES_API")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")

PRICE_CACHE_DIR = "data/price_cache"
os.makedirs(PRICE_CACHE_DIR, exist_ok=True)

# Configuration
QUERIES = {
    "whales": ("6395391", "dune_whales_cache.json", "whale_ml_ready.csv"),
    "market_intent": ("6385600", "dune_intent_cache.json", "market_intent_ml_ready.csv")
}
REQUEST_DELAY = 0.5

- Data Collection - Fetch Whale Data from Dune

In [17]:
def fetch_dune_incremental(query_id, cache_file):
    
    headers = {"x-dune-api-key": DUNE_API_KEY}
    today = pd.Timestamp(datetime.utcnow().date())
    
    # Load cache
    df_cached, fetch_start = pd.DataFrame(), None
    if os.path.exists(cache_file):
        cache = json.load(open(cache_file))
        df_cached = pd.DataFrame(cache["data"])
        df_cached["block_date"] = pd.to_datetime(df_cached["block_date"])
        fetch_start = pd.to_datetime(cache["last_block_date"]) + timedelta(days=1)
        
        if fetch_start >= today:
            print(f"✅ {cache_file} up-to-date")
            return df_cached
        print(f"🔄 Fetching {fetch_start.date()} → {today.date()}")
    else:
        print(f"🆕 Full fetch for {cache_file}")
    
    # Execute query
    payload = {"query_parameters": [{"name": "start_date", "type": "date", "value": fetch_start.strftime("%Y-%m-%d")}]} if fetch_start is not None else {}
    exec_id = requests.post(f"https://api.dune.com/api/v1/query/{query_id}/execute", 
                           headers=headers, json=payload).json()["execution_id"]
    
    # Poll until complete
    while True:
        status = requests.get(f"https://api.dune.com/api/v1/execution/{exec_id}/status", 
                            headers=headers).json()["state"]
        if status == "QUERY_STATE_COMPLETED": break
        if status == "QUERY_STATE_FAILED": raise RuntimeError("Query failed")
        time.sleep(10)
    
    # Get results
    df_new = pd.DataFrame(requests.get(f"https://api.dune.com/api/v1/execution/{exec_id}/results", 
                                       headers=headers).json()["result"]["rows"])
    
    if df_new.empty:
        return df_cached
    
    df_new["block_date"] = pd.to_datetime(df_new["block_date"])
    df_all = pd.concat([df_cached, df_new[df_new["block_date"] < today]], 
                      ignore_index=True).drop_duplicates("block_date", keep="last").sort_values("block_date")
    
    # Save cache
    json.dump({"last_block_date": df_all["block_date"].max().strftime("%Y-%m-%d"),
               "data": json.loads(df_all.to_json(orient="records", date_format="iso"))}, 
              open(cache_file, "w"))
    
    print(f"✅ {cache_file} updated ({len(df_all)} rows)")
    return df_all

In [18]:
# Load Dune Data (Whales & Market Intent)

# %%
print("="*60)
print("LOADING DUNE DATA")
print("="*60)

datasets = {}
for name, (qid, cache, output) in QUERIES.items():
    print(f"\n{name.upper()}")
    print("-"*40)
    datasets[name] = fetch_dune_incremental(qid, cache)
    datasets[name].to_csv(output, index=False)
    print(f"💾 Saved to {output}")
    time.sleep(0.5)

df_whales = datasets["whales"]
df_market_intent = datasets["market_intent"]

print(f"\n✅ Dune Data Loaded:")
print(f"   • Whales: {len(df_whales)} rows")
print(f"   • Market Intent: {len(df_market_intent)} rows")

# Display sample
print(f"\n📊 Whales Sample:")
display(df_whales.head(3))

LOADING DUNE DATA

WHALES
----------------------------------------
✅ dune_whales_cache.json up-to-date
💾 Saved to whale_ml_ready.csv



MARKET_INTENT
----------------------------------------
✅ dune_intent_cache.json up-to-date
💾 Saved to market_intent_ml_ready.csv

✅ Dune Data Loaded:
   • Whales: 1095 rows
   • Market Intent: 1095 rows

📊 Whales Sample:


,block_date,deposit_tx_count,deposit_withdrawal_ratio,exchange_volume_ratio,mega_whale_ratio,mega_whale_tx_count,mega_whale_volume_eth,net_flow_ma7,non_exchange_ratio,non_exchange_tx_count,non_exchange_volume_eth,std_whale_tx_size_eth,whale_exchange_deposits_eth,whale_exchange_withdrawals_eth,whale_net_exchange_flow_eth,whale_tx_count,whale_volume_eth,withdrawal_tx_count
0,2022-12-24,11,1.5554,0.2899,0.9475,36,108354.7154,-7203.9190,0.7101,24,81206.9408,3279.562931,20175.0838,12971.1648,-7203.9190,42,114353.1894,7
1,2022-12-25,10,4.4874,0.6327,1.0000,36,147540.1401,-33266.8544,0.3673,18,54184.9701,8106.493990,76342.4799,17012.6900,-59329.7899,36,147540.1401,8
2,2022-12-26,8,0.1890,0.3896,0.9751,50,196152.9223,-4359.6484,0.6104,38,122784.2329,7771.982946,12456.6605,65911.4244,53454.7638,55,201152.3179,9


- Coingecko Price Function

In [19]:
def fetch_coingecko_price_chunked(cg_id, start_date, end_date, api_key, chunk_days=30):
    """
    Fetch historical price data from CoinGecko in chunks.
    
    Using 30-day chunks for reliability with Pro API.
    """
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Use Pro API URL if API key is provided
    base_url = "https://pro-api.coingecko.com/api/v3" if api_key else "https://api.coingecko.com/api/v3"
    
    all_data = []
    current_start = start_dt
    
    while current_start < end_dt:
        current_end = min(current_start + timedelta(days=chunk_days), end_dt)
        
        from_ts = int(current_start.timestamp())
        to_ts = int(current_end.timestamp())
        
        url = f"{base_url}/coins/{cg_id}/market_chart/range"
        params = {"vs_currency": "usd", "from": from_ts, "to": to_ts}
        headers = {"x-cg-pro-api-key": api_key} if api_key else {}
        
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.get(url, params=params, headers=headers)
                response.raise_for_status()
                
                data = response.json()
                prices = data.get("prices", [])
                
                if prices:
                    all_data.extend(prices)
                
                print(f"   📥 {current_start.date()} → {current_end.date()} ({len(prices)} points)")
                
                # Rate limiting
                time.sleep(0.3)
                break  # Success, exit retry loop
                
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    wait_time = 60 * (attempt + 1)
                    print(f"   ⚠️  Rate limited. Waiting {wait_time}s... (attempt {attempt+1}/{max_retries})")
                    time.sleep(wait_time)
                    continue
                else:
                    # Print detailed error info
                    print(f"   ❌ Error: {e.response.status_code} - {e.response.text}")
                    print(f"   URL: {url}")
                    print(f"   Params: {params}")
                    
                    if attempt < max_retries - 1:
                        print(f"   Retrying in 5s... (attempt {attempt+1}/{max_retries})")
                        time.sleep(5)
                    else:
                        raise
        
        current_start = current_end + timedelta(days=1)
    
    if not all_data:
        return pd.DataFrame(columns=["date", "price"])
    
    df = pd.DataFrame(all_data, columns=["timestamp", "price"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms").dt.date
    df = df.groupby("date")["price"].mean().reset_index()
    df["date"] = pd.to_datetime(df["date"])
    
    symbol = cg_id.upper()
    print(f"📈 {symbol}: {len(df)} days | ${df['price'].min():.0f} - ${df['price'].max():.0f}")
    
    return df


In [20]:
def load_cached_price(symbol):
    """Load price data from cache."""
    path = f"{PRICE_CACHE_DIR}/{symbol}.csv"
    if os.path.exists(path):
        df = pd.read_csv(path)
        df["date"] = pd.to_datetime(df["date"])
        return df
    return None


def save_price_cache(symbol, df):
    """Save price data to cache."""
    path = f"{PRICE_CACHE_DIR}/{symbol}.csv"
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    df.sort_values("date").drop_duplicates("date").to_csv(path, index=False)


In [22]:
def get_price_data_incremental(symbol, cg_id, start_date, end_date):
    
    cached = load_cached_price(symbol)
    
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    today = pd.Timestamp(datetime.utcnow().date())
    yesterday = today - timedelta(days=1)
    
    if cached is None:
        print(f"📦 No cache for {symbol.upper()}. Fetching full range...")
        fetch_start = start_date
        base_df = pd.DataFrame()
    else:
        last_cached_date = pd.Timestamp(cached["date"].max())
        
        # Check if we have data through yesterday
        if last_cached_date >= yesterday:
            print(f"✅ {symbol.upper()} cache up-to-date (through {last_cached_date.date()})")
            return cached
        
        fetch_start = last_cached_date + timedelta(days=1)
        base_df = cached
        print(f"🔄 Updating {symbol.upper()}: {fetch_start.date()} → {yesterday.date()}")
    
    # Cap end_date at yesterday (we predict next day, so only need up to yesterday)
    end_date = min(end_date, yesterday)
    
    # Fetch historical data with chunking
    if fetch_start <= end_date:
        new_data = fetch_coingecko_price_chunked(
            cg_id,
            fetch_start.strftime("%Y-%m-%d"),
            end_date.strftime("%Y-%m-%d"),
            COINGECKO_API_KEY
        )
        
        df_new = new_data.rename(columns={"price": f"{symbol}_price"})
        df_all = pd.concat([base_df, df_new], ignore_index=True)
        save_price_cache(symbol, df_all)
    else:
        df_all = base_df
    
    return df_all

In [23]:
whales_min = df_whales['block_date'].min()
whales_max = df_whales['block_date'].max()
intent_min = df_market_intent['block_date'].min()
intent_max = df_market_intent['block_date'].max()

# Add 100-day buffer for technical indicators
min_date = min(whales_min, intent_min) - timedelta(days=100)
max_date = max(whales_max, intent_max)

print(f"\n📅 Date range: {min_date.date()} → {max_date.date()}")
print(f"   (Includes 100-day buffer for indicators)")

# Fetch ETH prices
print(f"\n{'='*40}")
print("ETHEREUM (ETH)")
print(f"{'='*40}")
df_eth = get_price_data_incremental("eth", "ethereum", min_date, max_date)
print(f"💾 ETH prices: {len(df_eth)} days")

# Fetch BTC prices
print(f"\n{'='*40}")
print("BITCOIN (BTC)")
print(f"{'='*40}")
df_btc = get_price_data_incremental("btc", "bitcoin", min_date, max_date)
print(f"💾 BTC prices: {len(df_btc)} days")



📅 Date range: 2022-09-15 → 2025-12-22
   (Includes 100-day buffer for indicators)

ETHEREUM (ETH)
🔄 Updating ETH: 2025-12-22 → 2025-12-22
💾 ETH prices: 1156 days

BITCOIN (BTC)
🔄 Updating BTC: 2025-12-22 → 2025-12-22
💾 BTC prices: 1156 days


In [24]:
print(f"\n📊 Loaded Datasets:")
print(f"   • Whales Features:     {len(df_whales):>5} rows")
print(f"   • Market Intent:       {len(df_market_intent):>5} rows")
print(f"   • ETH Prices:          {len(df_eth):>5} rows")
print(f"   • BTC Prices:          {len(df_btc):>5} rows")

print(f"\n📅 Date Ranges:")
print(f"   • Whales:        {df_whales['block_date'].min().date()} → {df_whales['block_date'].max().date()}")
print(f"   • Market Intent: {df_market_intent['block_date'].min().date()} → {df_market_intent['block_date'].max().date()}")
print(f"   • ETH:           {df_eth['date'].min().date()} → {df_eth['date'].max().date()}")
print(f"   • BTC:           {df_btc['date'].min().date()} → {df_btc['date'].max().date()}")

print("\n✅ All data ready for feature engineering and modeling!")

# Display samples
print(f"\n{'='*60}")
print("ETH PRICE SAMPLE")
print(f"{'='*60}")
display(df_eth.tail(5))

print(f"\n{'='*60}")
print("BTC PRICE SAMPLE")
print(f"{'='*60}")
display(df_btc.tail(5))


📊 Loaded Datasets:
   • Whales Features:      1095 rows
   • Market Intent:        1095 rows
   • ETH Prices:           1156 rows
   • BTC Prices:           1156 rows

📅 Date Ranges:
   • Whales:        2022-12-24 → 2025-12-22
   • Market Intent: 2022-12-24 → 2025-12-22
   • ETH:           2022-09-15 → 2025-12-21
   • BTC:           2022-09-15 → 2025-12-21

✅ All data ready for feature engineering and modeling!

ETH PRICE SAMPLE


,date,eth_price
1151,2025-12-17,2906.573711
1152,2025-12-18,2853.067689
1153,2025-12-19,2935.880471
1154,2025-12-20,2979.478368
1155,2025-12-21,2980.317996



BTC PRICE SAMPLE


,date,btc_price
1151,2025-12-17,86887.683596
1152,2025-12-18,86679.145619
1153,2025-12-19,87362.632014
1154,2025-12-20,88174.588995
1155,2025-12-21,88260.199788


- Feature Engineering from version 1

In [ ]:
def add_price_features(df, price_col, prefix):
    """Add price-based ML features"""
    df = df.sort_values('block_date').reset_index(drop=True)
    
    # Returns
    df[f'{prefix}_daily_return'] = df[price_col].pct_change()
    df[f'{prefix}_log_return'] = np.log(df[price_col] / df[price_col].shift(1))
    
    # Moving averages
    df[f'{prefix}_ma7'] = df[price_col].rolling(7, min_periods=1).mean()
    df[f'{prefix}_ma30'] = df[price_col].rolling(30, min_periods=1).mean()
    
    # Momentum
    df[f'{prefix}_vs_ma7'] = df[price_col] / df[f'{prefix}_ma7']
    df[f'{prefix}_vs_ma30'] = df[price_col] / df[f'{prefix}_ma30']
    
    # Volatility
    df[f'{prefix}_vol7'] = df[f'{prefix}_daily_return'].rolling(7, min_periods=1).std()
    df[f'{prefix}_vol30'] = df[f'{prefix}_daily_return'].rolling(30, min_periods=1).std()
    
    # Returns
    df[f'{prefix}_ret7d'] = df[price_col].pct_change(7)
    df[f'{prefix}_ret30d'] = df[price_col].pct_change(30)
    
    # RSI
    returns = df[f'{prefix}_daily_return']
    gains = returns.where(returns > 0, 0).rolling(14, min_periods=1).mean()
    losses = -returns.where(returns < 0, 0).rolling(14, min_periods=1).mean()
    rs = gains / (losses + 1e-10)
    df[f'{prefix}_rsi'] = 100 - (100 / (1 + rs))
    
    # Lags
    for lag in [1, 3, 7]:
        df[f'{prefix}_ret_lag{lag}'] = df[f'{prefix}_daily_return'].shift(lag)
    
    return df


def add_correlation_features(df):
    """Add ETH-BTC correlation features"""
    df['eth_btc_ratio'] = df['eth_price'] / df['btc_price']
    df['eth_btc_ratio_ma7'] = df['eth_btc_ratio'].rolling(7, min_periods=1).mean()
    df['eth_btc_corr_30d'] = df['eth_daily_return'].rolling(30, min_periods=20).corr(df['btc_daily_return'])
    df['eth_outperformance'] = df['eth_daily_return'] - df['btc_daily_return']
    return df

def create_target(df):
    """Create target: next day price direction"""
    df['next_day_return'] = df['eth_price'].pct_change().shift(-1)
    df['next_day_price_direction'] = (df['next_day_return'] > 0).astype(int)
    return df

# %%
print("\n⚙️ Engineering features...")

df_merged = add_price_features(df_merged, 'eth_price', 'eth')
df_merged = add_price_features(df_merged, 'btc_price', 'btc')
df_merged = add_correlation_features(df_merged)
df_merged = create_target(df_merged)

print(f"✅ Features created: {len(df_merged.columns)} total columns")

# Drop rows with NaN target
df_final = df_merged.dropna(subset=['next_day_price_direction']).copy()

print(f"✅ Final dataset: {len(df_final)} rows")
print(f"   Dropped {len(df_merged) - len(df_final)} rows (NaN target)")

# Save
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"💾 Saved: {OUTPUT_FILE}")



⚙️ Engineering features...
✅ Features created: 65 total columns
✅ Final dataset: 1095 rows
   Dropped 0 rows (NaN target)
💾 Saved: whale_prices_ml_ready.csv
